In [7]:
# Dados da loja
import pandas as pd
import json
import requests

data = {'Id': [27779,7606,14332,21357,30460], 
        'Store': [506,983,823,1060,646], 
        'DayOfWeek': [7,3,2,1,4], 
        'Date': ['2015-08-16','2015-09-09','2015-09-01','2015-08-24','2015-08-13'], 
        'Open': [0.0,1.0,1.0,1.0,1.0], 
        'Promo':[0,0,1,0,0], 
        'StateHoliday':[0,0,0,0,0],
        'SchoolHoliday':[0,0,0,0,1]}
data = pd.DataFrame(data)

# Dados de teste
data_raw = {'Store': [506,983,823,1060,646], 
            'StoreType': ['a','a','a','a','a'], 
            'Assortment': ['a','a','c','a','c'], 
            'CompetitionDistance': [220.0,620.0,16210.0,40.0,3430.0], 
            'CompetitionOpenSinceMonth':[2.0,9.0,11.0,3.0,float('NaN')], 
            'CompetitionOpenSinceYear':[2002.0,2014.0,2010.0,2014.0,float('NaN')],
            'Promo2':[1,0,0,1,1],
            'Promo2SinceWeek':[37.0,float('NaN'),float('NaN'),1.0,31.0],
            'Promo2SinceYear':[2009.0,float('NaN'),float('NaN'),2014.0,2013.0],
            'PromoInterval':['Jan,Apr,Jul,Oct',float('NaN'),float('NaN'),'Jan,Apr,Jul,Oct','Feb,May,Aug,Nov']}
data_raw = pd.DataFrame(data_raw)

# Merge dos dados de test + dados da loja
df_test = pd.merge( data, data_raw, how='left', on = 'Store' )

# Escolhendo loja para predição
df_test = df_test[df_test['Store'].isin([506,983,823,1060,646])]

# Removendo dias fechados
df_test = df_test[df_test['Open'] != 0]
df_test = df_test[~df_test['Open'].isnull()]
df_test = df_test.drop( 'Id', axis=1 )

# Passando dados para json
data = json.dumps( df_test.to_dict( orient='records' ) )

# Chamada da API
#url = 'http://0.0.0.0:5000/rossmann/predict'
url = 'https://rossmann-sales-predict-model.herokuapp.com/rossmann/predict'
header = {'Content-type': 'application/json'}
data = data
r = requests.post( url, data=data, headers=header )
#print( 'Status Code {}'.format( r.status_code ) )

# Convertendo dados recebidos da API para pandas DataFrame
d1 = pd.DataFrame( r.json(), columns = r.json()[0].keys() )

# Agrupando lojas 
d2 = d1[['store', 'prediction']].groupby( 'store' ).sum().reset_index()

for i in range( len(d2) ):
    print( 'Loja número {} irá vender R${:,.2f}  nas próximas 6 semanas'.format(
            d2.loc[i, 'store'],
            d2.loc[i, 'prediction'] ) )

Loja número 646 irá vender R$5,051.41  nas próximas 6 semanas
Loja número 823 irá vender R$7,709.86  nas próximas 6 semanas
Loja número 983 irá vender R$4,074.91  nas próximas 6 semanas
Loja número 1060 irá vender R$6,615.52  nas próximas 6 semanas
